In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
train_data = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test')

print("Training texts:", len(train_data.data))
print("Test texts:", len(test_data.data))

Training texts: 11314
Test texts: 7532


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# Pasamos el fichero a una lista (una línea por item)
with open('words.txt') as f:
    dictionary = f.read().splitlines()

In [5]:
vectorizer = CountVectorizer(binary= False, vocabulary=dictionary, stop_words='english', ngram_range = (1,1))

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
dir(train_data)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [8]:
train_data.target[0:100]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4,  8, 19,  4, 14,  6,  0,  1,
        7, 12,  5,  0, 10,  6,  2,  4,  1, 12,  9, 15,  7,  6, 13, 12, 17,
       18, 10,  8, 11,  8, 16,  9,  4,  3,  9,  9,  4,  4,  8, 12, 14,  5,
       15,  2, 13, 17, 11,  7, 10,  2, 14, 12,  5,  4,  6,  7,  0, 11, 16,
        0,  6, 17,  7, 12,  7,  3, 12, 11,  7,  2,  2,  0, 16,  1,  2,  7,
        3,  2,  1, 10, 12, 12, 17, 12,  2,  8,  8, 18,  5,  0,  1])

In [9]:
set(train_data.target)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

In [10]:
set(train_data.target_names)

{'alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc'}

Esto es para saber que hay suficientes elementos de cada tipo, por lo que podremos hacer selecciones aleatorias hasta tener 3 de cada

In [11]:
import numpy as np

In [12]:
unique, counts = np.unique(train_data.target, return_counts=True)
dict(zip(unique, counts))

{0: 480,
 1: 584,
 2: 591,
 3: 590,
 4: 578,
 5: 593,
 6: 585,
 7: 594,
 8: 598,
 9: 597,
 10: 600,
 11: 595,
 12: 591,
 13: 594,
 14: 593,
 15: 599,
 16: 546,
 17: 564,
 18: 465,
 19: 377}

In [13]:
train_data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [14]:
import numpy as np
import numpy.ma as ma

def write_terms (feature_names, data, vector_data, index):
    '''
    Escribe los términos presentes en un mensaje representado como bolsa de palabras.
    
    - feature_names: terminos usados para vectorizar
    - data: lista de mensajes original (si data==None no se muestra el mensaje original)
    - vector_data: matriz (dispersa) de mensaje vectorizados
    - index: posición del mensaje a mostrar
    '''
    # máscara para seleccionar sólo el mensaje en posición index
    mask=vector_data[index,:]>0
    
    # términos que aparecen en ese mensaje vectorizado
    terminos = ma.array(feature_names, mask = ~(mask[0].toarray()))
    
    # mostrar mensaje original
    if data:
        print('Mensaje', index, ':', data[index])
    
    # mostrar términos que aparecen en el mensaje vectorizado
    print('Mensaje', index, 'vectorizado:', terminos.compressed(),'\n')

In [50]:
train_vector_data = vectorizer.fit_transform(train_data.data)
test_vector_data = vectorizer.fit_transform(test_data.data)

In [16]:
print(train_vector_data)

  (0, 4151)	1
  (0, 52160)	1
  (0, 53807)	1
  (0, 56907)	1
  (0, 78575)	1
  (0, 112523)	1
  (0, 112565)	1
  (0, 124009)	1
  (0, 124372)	1
  (0, 149051)	1
  (0, 177802)	1
  (0, 184493)	1
  (0, 185287)	1
  (0, 190955)	1
  (0, 208543)	1
  (0, 213043)	1
  (0, 218699)	1
  (0, 221810)	1
  (0, 221819)	1
  (0, 225798)	1
  (0, 242690)	1
  (0, 251793)	1
  (0, 271493)	1
  (0, 306014)	1
  (0, 314710)	1
  :	:
  (11313, 12051)	1
  (11313, 31047)	1
  (11313, 110477)	1
  (11313, 124009)	1
  (11313, 163078)	1
  (11313, 170696)	1
  (11313, 192874)	1
  (11313, 217697)	1
  (11313, 218699)	1
  (11313, 239824)	1
  (11313, 265072)	2
  (11313, 271493)	1
  (11313, 306014)	1
  (11313, 340787)	1
  (11313, 359935)	1
  (11313, 360545)	1
  (11313, 365530)	1
  (11313, 385115)	2
  (11313, 389038)	1
  (11313, 391937)	1
  (11313, 398618)	1
  (11313, 400462)	1
  (11313, 410906)	1
  (11313, 413140)	1
  (11313, 419956)	1


In [17]:
feature_names = vectorizer.get_feature_names()

In [18]:
print(feature_names[265072])

number


In [19]:
write_terms(feature_names, None, train_vector_data, 1)

Mensaje 1 vectorizado: ['acceleration' 'adapters' 'answered' 'article' 'attained' 'brave' 'cards'
 'clock' 'days' 'detailing' 'disk' 'especially' 'experiences' 'final'
 'floppy' 'floppies' 'functionality' 'heat' 'hour' 'keywords' 'knowledge'
 'lines' 'message' 'network' 'number' 'organization' 'oscillator'
 'posting' 'procedure' 'rated' 'reports' 'requested' 'send' 'shared'
 'sinks' 'souls' 'speed' 'subject' 'summary' 'summarizing' 'thanks'
 'upgrade' 'upgraded' 'usage'] 



In [20]:
type(train_vector_data)

scipy.sparse.csr.csr_matrix

Procedemos a elegir 3 mensajes de cada tipo

In [21]:
import random

In [22]:
len(test_data.target)

7532

In [23]:
test_data.target[103]

15

In [24]:
#while the last element of the sum of all the previous elements (which is the sum of the total selectec texts) 
#    is less than the desired amount, it will keep choosing candidates

selected = [0]*len(test_data.target_names)

number_samples = 3

selected_index = [0]*len(test_data.target_names)*number_samples

while (np.cumsum(selected)[-1] < len(selected)*number_samples):

    #Indice aleatorio de la lista de mensajes
    index = random.randint(0, len(test_data.target))
    #cat = numero de la categoria a la que pertenece el índice elegido
    cat = test_data.target[index]
    #Comprobamos que no haya ya 3 elementos de la categoría a la que pertenezca el index y lo añadimos a la cuenta y a la lista de índices
    #La lista de índices estará ordenada por categorías
    if(selected[cat] < 3):
        selected_index[cat*number_samples + selected[cat]] = index
        selected[cat] += 1

In [25]:
selected

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [26]:
np.cumsum(selected)[-1]

60

In [27]:
len(selected_index)

60

Igual merece más la pena ponerlo en una matriz de N*M siendo N el número de elementos por cada clase y M el número de clases

In [28]:
selected_index

[95,
 3112,
 4853,
 3495,
 4330,
 7337,
 3467,
 7041,
 6012,
 4711,
 7094,
 3095,
 6972,
 6319,
 6646,
 3922,
 1933,
 2337,
 5298,
 3106,
 1437,
 2547,
 3791,
 4832,
 2512,
 1291,
 6637,
 1350,
 6346,
 1700,
 601,
 7034,
 2616,
 5923,
 3309,
 1215,
 7507,
 4569,
 6152,
 4630,
 5793,
 2137,
 7112,
 3732,
 1621,
 6786,
 3761,
 6729,
 4675,
 4772,
 877,
 7101,
 1318,
 6138,
 1675,
 6871,
 4342,
 4635,
 2300,
 6690]

In [174]:
new_shape = np.reshape(selected_index, [20,3])

In [175]:
print(new_shape)

[[  95 3112 4853]
 [3495 4330 7337]
 [3467 7041 6012]
 [4711 7094 3095]
 [6972 6319 6646]
 [3922 1933 2337]
 [5298 3106 1437]
 [2547 3791 4832]
 [2512 1291 6637]
 [1350 6346 1700]
 [ 601 7034 2616]
 [5923 3309 1215]
 [7507 4569 6152]
 [4630 5793 2137]
 [7112 3732 1621]
 [6786 3761 6729]
 [4675 4772  877]
 [7101 1318 6138]
 [1675 6871 4342]
 [4635 2300 6690]]


In [176]:
len(new_shape)

20

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
type(train_vector_data[2252])

scipy.sparse.csr.csr_matrix

In [31]:
#indices = selected_index[0:3]
#print(indices)
#out1 = train_vector_data.tocsc()[indices,:]

In [32]:
cosSim1 = cosine_similarity(train_vector_data[2252], train_vector_data)

A ver, entonces, lo que creo que hay que hacer es:

    1) Hacer el cosine similarity para cada una de las 60 indices que hemos elegido.
    2) Agruparlos de 3 en 3 ya que es el número de muestras que nos dicen que cojamos (ponerlo como un parámetro)
    3) Ordenar los 20 arrays, en total cada uno tendrá (número de textos)*3 elementos
    4) Cuando tengamos todos los elementos ordenador hacemos lo de la exhaustividad (recall, es decir de entre todos los que han dado positivo (que se parecen al texto objetivo) cuantos son realmente de la misma categoría(True Pos/(True Pos + False Neg))) entre valores de 3 y 10. Eso significa que tenemos que ver para los mejores X valores (entre 3 y 10) cuantos son de la misma categoría que la clase que estamos comparando
    5) Hacer la media de los 

Nononono a ver

Lo que piden de:  la precisión de la lista de resultados con nivel de exhaustividad 3 y 10.

Lo que hay que hacer es: De los X valores que más se parezcan entre los X primeros recuperados. Por lo que símplemente es coger y decir: Si para un nivel de exhaustividad 7, cogemos los 7 elementos que más se parezcan a los textos que hemos metido, después comprobamos de esos 7 cuantos son de la clase a la que perteneces los mensajes seleccionados antes.

Entonces, después cogemos esos 8 resultados (los valores de 3 a 10) y hacemos la media. Eso para cada clase.

Así podemos ver cual es la clase que más parecidos tiene o algo así (?)


Y luego hay que hacer lo mismo pero con el TF-IDF

la duda que luego voy a preguntar es que si lo de exhaustivodad de nivel 5 por ejemplo, significa realmente que solo hay que coger los 5 elementos más TOP, me parece raro porque hay miles y miles y quedarse con tan pocos se me hace raro

Intentar hacer la movida con funciones para que quede guay y bien indicado

In [33]:
cosSim1.shape

(1, 11314)

In [34]:
cosSim2 = cosine_similarity(train_vector_data[2251], train_vector_data)

In [35]:
cosSim3 = cosine_similarity(train_vector_data[2250], train_vector_data)

In [36]:
cosSim = np.concatenate((cosSim1, cosSim2, cosSim3), axis=1)

In [37]:
type(cosSim)

numpy.ndarray

In [38]:
cosSim = -np.sort(-cosSim)

In [39]:
cosSimFinal = cosSim[0]

In [64]:
cosSimFinal.shape

(33942,)

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [394]:
x = np.array([0.254864, 0.4873, 0.742])

print(-np.sort(-x))

[0.742    0.4873   0.254864]


In [387]:
def sortSimilarityIndex(cosSim):
    
    for i, category in enumerate(cosSim):
        for j, sim in enumerate(category):
            cosSim[i][j] = np.argsort(-sim)
    return cosSim

In [386]:
def cosSim_calc2(indexes, test_vector, train_vector):
    cosSim = np.zeros(indexes.shape, object)
    for i, indexList in enumerate(indexes):
        for j, index in enumerate(indexList):
            cosSim[i][j] = cosine_similarity(test_vector[index], train_vector)[0]
    return cosSim

In [397]:
cos_sim2 = cosSim_calc2(new_shape,test_vector_data, train_vector_data)

In [339]:
len(cos_sim2[0][0])

11314

In [398]:
cos = sortSimilarityIndex(cos_sim2)

In [351]:
train_data.target[11117]

15

In [382]:
cos[0][0][11117]

0.2525934760075915

In [399]:
print(cos)

[[array([11117,  7260,  9430, ...,   144,  1541,  9412], dtype=int64)
  array([ 500,  335, 8710, ..., 1541, 3382, 9412], dtype=int64)
  array([1889, 3528, 6884, ...,  144, 9412, 1541], dtype=int64)]
 [array([ 7359, 10907,  4166, ...,  1541,  5829,  9412], dtype=int64)
  array([10226,  4583,  8251, ...,   144,  1541,  9412], dtype=int64)
  array([ 5405, 11082,  1041, ...,   144,  1541,  9412], dtype=int64)]
 [array([2140, 6511, 4817, ..., 4582,  144, 9412], dtype=int64)
  array([ 171, 1603,  820, ..., 8519,  144, 1541], dtype=int64)
  array([11232,  8484,  3825, ...,  3629,  8519,  9412], dtype=int64)]
 [array([1585, 1677, 2897, ..., 1541, 9412, 8519], dtype=int64)
  array([1339,  685,  943, ..., 4582, 9412, 3382], dtype=int64)
  array([ 3972,  6796,  7961, ...,  3629,  9412, 10433], dtype=int64)]
 [array([11232, 10451,   776, ...,   144,  1541,  9412], dtype=int64)
  array([ 8303,  2735,  3048, ..., 10275,  1541,  9412], dtype=int64)
  array([ 3588,  8851,  1960, ..., 10275,  1541,  94

In [274]:
cos[0][0][0]

array([0.25259348, 0.23823145, 0.23426064, ..., 0.0045243 , 0.00406627,
       0.00332009])

In [65]:
#Funcion que le metes los índices seleccionados, el número de elementos por categoría y/o  el número de categorías
#Devuelve un array con un número de columnas igual al número de categorías y número de filas igual al
#    (número de textos - 1)*número de elementos elegidos por categoría - 3. El - 3 es para quitar las comparaciones de los textos
#    con ellos mismos. La lista está completamente ordenada de mayor a menor.

def cosSim_calc(indexes, elem, rows, numCat = None):
    print(indexes)
    print(len(indexes))
    
    cols = int(len(indexes)/elem)
    print(rows, cols)
    
    sim = [[0]*rows]*cols
    
    col = 0
    tempBatch_index = []
    #Para cada indice seleccionado hacemos la cosine_similarity
    for index in indexes:
        
        #Cogemos un grupo de "elem" elementos, de esta manera tendremos los indices con los que vamos a trabajar a continuación
        tempBatch_index.append(index)
        
        #Si ya hemos cogido los "elem" elementos, procedemos a usarlos
        if(len(tempBatch_index) == 3):
            #print(tempBatch_index)
            #Los agrupamos por categorías
            agg = []
            print(col)
            for i in range(elem):
                cosSim = cosine_similarity(test_vector_data[tempBatch_index[i]], train_vector_data)
                agg = np.concatenate((agg, cosSim[0]))
                #print(-np.sort(-agg))
                
            #Transformamos la representación de la información a dataFrames para poder mantener tener la categoría de cada elemento
            info = pd.DataFrame(agg, columns=['puntuacion'])
            #juntamos los indices para que coincidan con la información del DataFrame
            categorias = np.array([])
            for i in range(elem):
                categorias = np.append(categorias, train_data.target)
            
            info['categoria'] = categorias
            info = info.sort_values(by='puntuacion', ascending=False)
            #info = info[elem:]
            sim[col] = info
            print(max(sim[col]['puntuacion']))
            #print(agg[:5])
            #sim[col] = -np.sort(-agg)
            #print(sim[col])
            #Ahora quitamos los elementos que son =1, es decir los que se han comparado con si mismos, que ahora están en las "elem"
            #    primeras posiciones
            #sim[col] = sim[col][elem:]
            
            #print(sim[col][:10])
            #print(max(sim[col]))
            #print(tempBatch_index)
            col+=1
            tempBatch_index = []

    
    
    return sim

In [66]:
cos_sim = cosSim_calc(selected_index, 3, len(train_data.data))

[95, 3112, 4853, 3495, 4330, 7337, 3467, 7041, 6012, 4711, 7094, 3095, 6972, 6319, 6646, 3922, 1933, 2337, 5298, 3106, 1437, 2547, 3791, 4832, 2512, 1291, 6637, 1350, 6346, 1700, 601, 7034, 2616, 5923, 3309, 1215, 7507, 4569, 6152, 4630, 5793, 2137, 7112, 3732, 1621, 6786, 3761, 6729, 4675, 4772, 877, 7101, 1318, 6138, 1675, 6871, 4342, 4635, 2300, 6690]
60
11314 20
0
0.44866998231315963
1
0.5940885257860047
2
0.45939149659322087
3
0.6761070917156279
4
0.4097999809317564
5
0.37363235887853663
6
0.9697651491183022
7
0.4313489850488289
8
0.6520506636966266
9
0.35148175122006836
10
0.386333704643128
11
0.7498063766587063
12
0.7540739028638984
13
0.31193682406898127
14
0.6079476701167376
15
0.5892556509887895
16
0.4140043409440132
17
0.6549612691572329
18
0.4813299149207704
19
0.4542199791661352


In [194]:
len(cos_sim[0])

33942

In [67]:
len(cos_sim[0])

33942

In [68]:
type(cos_sim)

list

In [70]:
print(max(cos_sim[13]['puntuacion'].values))

0.31193682406898127


Ahora toca hacer los niveles de exhaustividad con la información bien colocadita que tenemos aquí

Lo que vamos a hacer es dejar en una matriz de N*M los resultados del recall
    Siendo N=número de clases y M=niveles de exhaustividad

In [84]:
cos_sim[1]

,puntuacion,categoria
7359,0.594089,1.0
28033,0.521286,1.0
33710,0.440959,1.0
23669,0.434975,1.0
10907,0.424264,1.0
...,...,...
9412,0.006013,10.0
11458,0.005246,2.0
32040,0.005011,10.0
12855,0.004715,2.0


In [435]:
def nivelesExhaustividad(data, nivel_min = 3, nivel_max = 10):
    #exhaustN = np.zeros(data.shape, object)
    exhaust = []
    
    for nivel in range(nivel_min, nivel_max + 1):
        exhaustN = np.zeros(data.shape, object)
        for i, indexList in enumerate(data):
            for j, index in enumerate(indexList):
                exhaustN[i][j] = data[i][j][:nivel:1]
        exhaust.append(exhaustN)
        
        exhaustNP = np.array(exhaust)
        
    return exhaustNP

In [436]:
exhaustIndex = nivelesExhaustividad(cos)

In [437]:
print(exhaustIndex)

[[[array([11117,  7260,  9430], dtype=int64)
   array([ 500,  335, 8710], dtype=int64)
   array([1889, 3528, 6884], dtype=int64)]
  [array([ 7359, 10907,  4166], dtype=int64)
   array([10226,  4583,  8251], dtype=int64)
   array([ 5405, 11082,  1041], dtype=int64)]
  [array([2140, 6511, 4817], dtype=int64)
   array([ 171, 1603,  820], dtype=int64)
   array([11232,  8484,  3825], dtype=int64)]
  [array([1585, 1677, 2897], dtype=int64)
   array([1339,  685,  943], dtype=int64)
   array([3972, 6796, 7961], dtype=int64)]
  [array([11232, 10451,   776], dtype=int64)
   array([8303, 2735, 3048], dtype=int64)
   array([3588, 8851, 1960], dtype=int64)]
  [array([  12, 1943, 2371], dtype=int64)
   array([7687, 9507, 2848], dtype=int64)
   array([2695, 6769, 2799], dtype=int64)]
  [array([10555,  6541,  3163], dtype=int64)
   array([10291,  7816,  4960], dtype=int64)
   array([9160, 6815,  680], dtype=int64)]
  [array([ 4467,  6902, 10692], dtype=int64)
   array([2434, 8150, 3386], dtype=int64)


In [446]:
dir(train_data.)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [452]:
def calcPrecision(exhaustIndex, train_data):
    precision = np.zeros(exhaustIndex.shape, object)
    
    for i, nivel in enumerate(exhaustIndex):
        for j, indexList in enumerate(nivel):
            for k, BestIndexes in enumerate(indexList):
                prec = 0
                for index in BestIndexes:
                    #Si el indice del train data coincide con la j, es bueno
                    if(train_data[index] == j):
                        prec += 1
                prec /= len(BestIndexes)
                
                precision[i][j][k] = prec
                
    return precision

In [453]:
precision = calcPrecision(exhaustIndex, train_data.target)

In [454]:
precision

array([[[0.6666666666666666, 0.0, 1.0],
        [1.0, 1.0, 1.0],
        [0.3333333333333333, 0.6666666666666666, 0.3333333333333333],
        [1.0, 1.0, 0.6666666666666666],
        [0.0, 0.0, 0.0],
        [0.0, 1.0, 0.3333333333333333],
        [1.0, 1.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 1.0, 1.0],
        [1.0, 0.6666666666666666, 0.3333333333333333],
        [0.0, 0.0, 1.0],
        [1.0, 0.3333333333333333, 1.0],
        [0.6666666666666666, 0.3333333333333333, 0.6666666666666666],
        [0.6666666666666666, 0.3333333333333333, 1.0],
        [1.0, 0.3333333333333333, 0.0],
        [1.0, 0.6666666666666666, 1.0],
        [0.0, 0.6666666666666666, 0.0],
        [0.0, 1.0, 1.0],
        [0.0, 0.3333333333333333, 1.0],
        [0.0, 0.0, 0.0]],

       [[0.5, 0.0, 0.75],
        [0.75, 0.75, 1.0],
        [0.5, 0.75, 0.5],
        [1.0, 1.0, 0.75],
        [0.0, 0.25, 0.0],
        [0.0, 1.0, 0.25],
        [0.75, 0.75, 0.0],
        [0.0, 1.0, 0.0],
        [0.25, 1.0,

In [ ]:
def precMedia(precisiones):
    
    